In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab1/ai_scholar/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
import csv
import numpy as np
import sys
from pathlib import Path
import pickle

In [ ]:
scholar_all = np.load(GOOGLE_DRIVE_PATH+'general/gs_scholars_all_upd.npy', allow_pickle=True)

In [ ]:
scholar_all[1]

# table tab:scholar_analysis

## academic age & lifespan analysis

In [ ]:
total_url_dict, female_url_dict = {}, {}
bug_check = []
bug_paper_check = []
for scholar in scholar_all:
  papers = scholar['papers']
  gender = df.loc[df['url'] == scholar['url'], 'gender'].item()
  year_set = set()
  for paper in papers:
    if paper[5] == '':
      continue
    if int(paper[5]) <= 1950:
      bug_paper_check.append((scholar['url'], int(paper[5])))
      continue
    if int(paper[5]) > 2022:
      continue
    year_set.add(int(paper[5]))
  if len(year_set) == 0:
    continue
    # year_first, year_last = 2023, 2022
    # bug_check.append(scholar['url'])
  year_first, year_last = min(year_set), max(year_set)

  total_url_dict[scholar['url']] = (year_first, year_last)
  if gender == 'F':
    female_url_dict[scholar['url']] = (year_first, year_last)

In [ ]:
bug_check

In [ ]:
bug_paper_check

In [ ]:
total_aca_age = pd.Series([2022-year[0]+1 for year in total_url_dict.values()])
total_aca_span = pd.Series([year[1] - year[0]+1 for year in total_url_dict.values()])
female_aca_age = pd.Series([2022-year[0]+1 for year in female_url_dict.values()])
female_aca_span = pd.Series([year[1] - year[0]+1 for year in female_url_dict.values()])

In [ ]:
total_aca_age.describe()

In [ ]:
female_aca_age.describe()

In [ ]:
female_aca_span.describe()

In [ ]:
total_aca_span.describe()

## top coauthors analysis

In [ ]:
!pip install efficiency

In [ ]:
class Name2Gender:
    def __init__(self):
        self.full_name2gender = self._load_full_name_gender()
        self.first_name2gender = self._load_first_name_gender()

    def _load_first_name_gender(self):
        # 100711 lines
        from collections import defaultdict
        df = pd.read_csv(GOOGLE_DRIVE_PATH+"data/gender_preprocess/firstname_gender.csv")
        first_name_n_gender = df.to_dict(orient='records')
        first_name2gender = {dic['name']: dic['gender'] for dic in first_name_n_gender}
        return first_name2gender

    def _load_full_name_gender(self):
        def _file2full_name(file):
            from efficiency.log import fread
            full_names = fread(GOOGLE_DRIVE_PATH+'data/gender_preprocess/'+file, delete_empty=True, if_strip=True)
            # full_names = U.read_file(file)
            full_names = [' '.join(i.split(', ', 1)[::-1]).lower() for i in full_names]
            return full_names

        gender2file = {'M': 'acl-male.txt',
                       'F': 'acl-female.txt',
                       }
        full_name2gender = {}
        for gender, file in gender2file.items():
            full_names = _file2full_name(file)
            full_name2gender.update({i: gender for i in full_names})
        return full_name2gender

    def lookup_gender(self, full_name):
        gender = self.full_name2gender.get(full_name.lower(), '-')
        if gender == '-':
            first_name = full_name.rsplit(' ', 1)[0].lower()
            gender = self.first_name2gender.get(first_name, '-')
        return gender

In [ ]:
name2gender = Name2Gender()

In [ ]:
name2gender.lookup_gender('Christian A. Lang')

In [ ]:
print(scholar_all[1]['co_authors'])

In [ ]:
percent_list = []
for scholar in scholar_all: # scholar_all is np.load() from 78k_scholar_new
  if name2gender.lookup_gender(scholar['name']) == 'F':
      co_authors = scholar['co_authors']
      if co_authors:
          male_count, female_count = 0, 0
          for each in co_authors:
              # coauthor_names.append(each[1])
              if name2gender.lookup_gender(each[1]) == 'M':
                  male_count += 1
              elif name2gender.lookup_gender(each[1]) == 'F':
                  female_count += 1         
          if male_count + female_count != 0:
              female_percentage = female_count / (male_count + female_count)
          elif male_count == 0 and female_count != 0:
              female_percentage = 1
          elif female_count == 0 and male_count != 0:
              female_percentage = 0
          else:
              continue
          percent_list.append(female_percentage)
percent_series = pd.Series(percent_list)
percent_series.describe()

# scholar_centric analysis

## to dataframe

In [ ]:
# %%capture
import pandas as pd
df = pd.DataFrame.from_records(scholar_all)
df.head(1)

In [ ]:
df.head(5)

In [ ]:
# For each of the 78K scholars, “#co-authors/scholar_profile” report “avg+/- std”
for idx, row in df.iterrows():
  co_authors = row['co_authors']
  if co_authors is not None:
    df.at[idx, 'num_coauthors'] = len(co_authors)
  else:
    df.at[idx, 'num_coauthors'] = None

In [ ]:
df['num_coauthors']

In [ ]:
df['num_coauthors'].describe()

In [ ]:
df['num_coauthors'].isna().sum()

In [ ]:
# On average, report %co-authors can be found in gs_scholars.jsonl (⅕ or 1/10 missing)
print(df['num_coauthors'].isna().sum() / len(df['num_coauthors']))

In [ ]:
# Make sure that #co-authors do not differ #topics, otherwise we can normalized_topic_diversity = num_topics_among_all_coauthors/num_coauthors
for idx, row in df.iterrows():
  co_domain = row['domain_labels']
  if co_domain is not None:
    df.at[idx, 'num_domain'] = len(co_domain)
  else:
    df.at[idx, 'num_domain'] = None

In [ ]:
df['num_domain'].describe()

In [ ]:
# get normalized_topic_diversity
for idx, row in df.iterrows():
  co_author_list = row['co_authors']
  if co_author_list is None:
    df.at[idx, 'norm_topic_diversity'] = None
  else:
    sid_list = []
    for co_author in co_author_list:
      if 'user' in co_author[0]:
        sid_list.append(co_author[0].split('user=', 1)[1][:12])
    fields_set = set()
    for sid in sid_list:
      url = 'https://scholar.google.com/citations?hl=en&user='+sid
      if url in df['url'].values:
        domain = df.loc[df['url'] == url, 'domain_labels'].item()
        fields_set.update(set(domain))
    df.at[idx, 'norm_topic_diversity'] = len(fields_set)
# TODO: rename norm_topic_diversity to topic_diversity, compute norm_topic_diversity

In [ ]:
df['norm_topic_diversity']

In [ ]:
df = df.rename(columns={'norm_topic_diversity': 'top_topic_diversity', 'num_coauthors': 'top_num_coauthors'})

In [ ]:
df

In [ ]:
np.save(GOOGLE_DRIVE_PATH+'general/gs_scholars_all_upd.npy', df.to_dict('records'), allow_pickle=True)

## Another way of getting a full list of coauthors

In [ ]:
tmp_list = []
for scholar in scholar_all:
  for paper in scholar['papers']:
    if paper[1] == 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding':
      tmp_list.append(paper)
print(tmp_list)

In [ ]:
from collections import defaultdict
paper_title2scholars_list = defaultdict(list)
for scholar in scholar_all:
  gs_sid = scholar['gs_sid']
  papers = scholar['papers']
  for paper in papers:
    title = paper[1]
    paper_title2scholars_list[title].append(gs_sid)

In [ ]:
# count the number of all coauthors, including those are not in ai scholars
for scholar in scholar_all:
  num_coauthors_all_wwo_ai = 0
  coauth_set = set()
  for paper in scholar['papers']:
    coauth_item_set = set(paper[2][0].split(', '))
    coauth_set.update(coauth_item_set)
  if '...' in coauth_set:
    num_coauthors_all_wwo_ai = len(coauth_set) - 1
  else:
    num_coauthors_all_wwo_ai = len(coauth_set)
  scholar['num_coauthors_all_wwo_ai'] = num_coauthors_all_wwo_ai

In [ ]:
# add top coauthors to coauthors all
for scholar in scholar_all:
  coauth_set = set(scholar['co_authors_all'])
  if scholar['co_authors'] is not None:
    for coauthor in scholar['co_authors']:
      if 'user' in coauthor[0]:
        # print(set([coauthor[0].split('user=', 1)[1][:12]]))
        # break
        coauth_set.update(set([coauthor[0].split('user=', 1)[1][:12]]))
    scholar['co_authors_all'] = coauth_set


In [ ]:
scholar_all[1]['co_authors_all']

In [ ]:
with open(GOOGLE_DRIVE_PATH+'general/paper_title2scholars_list.pkl', 'wb') as file:
  pickle.dump(paper_title2scholars_list, file)

In [ ]:
with open(GOOGLE_DRIVE_PATH+'general/paper_title2scholars_list.pkl', 'rb') as file:
  paper_title2scholars_list = pickle.load(file)

In [ ]:
scholar_all[0]['papers'][1]

In [ ]:
df.head(2)

In [ ]:
# construct the co_authors_all entry (name, gs_sid)
for scholar in scholar_all:

  co_authors_gssid_set = set()
  for paper in scholar['papers']:
    # print(paper_title2scholars_list[paper[1]])
    co_authors_gssid_set.update(set(paper_title2scholars_list[paper[1]]))
  scholar['co_authors_all'] = co_authors_gssid_set
  # break

In [ ]:
scholar_all[0]['co_authors_all']

# new_cluster_data.jsonl (outdated)

*   列表项
*   列表项



In [ ]:
print(len(scholar_all))

In [ ]:
# if need sort:
def sort_by_cite(cluster_url):
  cluster_url.sort(key = lambda x: int(x['citation_table'][0]))
  return cluster_url[::-1]
sorted_scholars = sort_by_cite(scholar_all)

In [ ]:
# else:
sorted_scholars = scholar_all

In [ ]:
def get_field_by_index(index):
  return sorted_scholars[index]['domain_labels']

In [ ]:
def get_gssid_by_index(index):
  return sorted_scholars[index]['gs_sid']

In [ ]:
df_sorted_scholars = pd.DataFrame.from_records(sorted_scholars)
df_sorted_scholars

In [ ]:
df_all = pd.DataFrame.from_records(scholar_all)
df_all

In [ ]:
my_string="hello python worlda a , i'm a beginner "
print(my_string.split("world",1)[1])

In [ ]:
def get_field_by_gssid(sid):
  field_series = df_sorted_scholars.loc[df_sorted_scholars['gs_sid'] == sid, 'domain_labels']
  if field_series.empty:
    return []
  else:
    return field_series.item()

In [ ]:
# return a list of gs_sids of co_authors given sid
def get_coauthor_by_gssid(sid):
  co_author_list = df_all.loc[df_all['url'] == 'https://scholar.google.com/citations?hl=en&user='+sid, 'co_authors'].item()
  if co_author_list is None:
    return []
  sid_list = []
  for co_author in co_author_list:
    if 'user' in co_author[0]:
      sid_list.append(co_author[0].split('user=', 1)[1][:12])
  return sid_list

In [ ]:
get_coauthor_by_gssid(get_gssid_by_index(3))

In [ ]:
# scholar-centric analysis
def get_cofield_by_index(index):
  gssid = get_gssid_by_index(index)
  # while len(get_coauthor_by_gssid(gssid)) == 0:
  #   index += 1
  #   gssid = get_gssid_by_index(index)
  sid_list = get_coauthor_by_gssid(gssid)
  # add the author itself
  sid_list.append(gssid)
  # return all fields
  fields_set = set()
  # print(len(sid_list))
  for sid in sid_list:
    fields_set.update(set(get_field_by_gssid(sid)))
    # print(get_field_by_gssid(sid))
  return fields_set

In [ ]:
get_cofield_by_index(3)

In [ ]:
intpoint_array = np.arange(0, len(scholars), 1000)
avg_num_field = []
for intpoint in intpoint_array:
  num_field = []
  for point in range(intpoint, intpoint+100):
    num_field.append(len(get_cofield_by_index(point)))
  avg_num_field.append(np.mean(num_field, dtype=int))
avg_num_field

## plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
fig.canvas.draw()
ax.plot(np.arange(len(avg_num_field)), avg_num_field)
# ax.set_xticks([0, 20000, 40000, 68128])
ax.set_xticklabels([0, 0, 10000, 20000, 30000, 40000, 50000, 60000, 70000])
ax.set_xlabel('scholar rank sorted by total citations')
ax.set_ylabel('average number of fields')
plt.show()

# cluster analysis

## increase and then decrease

In [ ]:
# clusters 1, 6, 7 are increase and then decrease
import pickle
model_labels = []
with open(GOOGLE_DRIVE_PATH+'model_label.pkl', 'rb') as f:
    model_labels = pickle.load(f)
print(model_labels)

In [ ]:
len(model_labels) # should use scholars (new_cluster_data.jsonl)

In [ ]:
cluster1_idx_list, cluster6_idx_list, cluster7_idx_list = [], [], []
for i in range(len(model_labels)):
  if model_labels[i] == 1:
    cluster1_idx_list.append(i)
  elif model_labels[i] == 6:
    cluster6_idx_list.append(i)
  elif model_labels[i] == 7:
    cluster7_idx_list.append(i)

In [ ]:
# df_scholars = pd.DataFrame.from_records(scholars)
# df_scholars

In [ ]:
from collections import defaultdict
def gen_word_freq_dict(idx_list):
  word_freq_dict = defaultdict(int)
  for idx in idx_list:
    for label in scholars[idx]['domain_labels']:
      word_freq_dict[label] += 1
  word_freq_dict = {k: v for k, v in sorted(word_freq_dict.items(), key=lambda item: item[1], reverse=True)}
  return word_freq_dict

In [ ]:
gen_word_freq_dict(cluster1_idx_list)

In [ ]:
gen_word_freq_dict(cluster6_idx_list)

In [ ]:
gen_word_freq_dict(cluster7_idx_list)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def gen_wordcloud(freq_dict, cluster_num):
  freq_list = []
  for key, value in freq_dict.items():
    freq_list += value * [key]
  freq_str = ' '.join(freq_list)
  # print(freq_str)
  # generate wordcloud
  wordcloud = WordCloud(width=5000, height=4000, collocations=False, random_state=10).generate(freq_str)
  plt.figure(figsize=(10, 8))
  plt.imshow(wordcloud)
  plt.title(f'cluster {cluster_num}')
  plt.axis('off')
  plt.savefig(f'cluster_{cluster_num}.pdf')

In [ ]:
gen_wordcloud(gen_word_freq_dict(cluster7_idx_list), 7)

### whether they no longer publish any new paper

In [ ]:
tmp = df_all.loc[df_all['url'] == 'https://scholar.google.com/citations?hl=en&user=bSU7LYoAAAAJ']
tmp

In [ ]:
def get_lastyear_dict(idx_list):
  year_dict = defaultdict(int)
  bug_check = []
  bug_paper_check = []
  for scholar_idx in idx_list:
    sid = scholars[scholar_idx]['gs_sid']
    url = 'https://scholar.google.com/citations?hl=en&user='+sid
    scholar_series = df_all.loc[df_all['url'] == url]
    papers = scholar_series['papers'].item()
    year_set = set()
    for paper in papers:
      if paper[5] == '':
        continue
      if int(paper[5]) <= 1950:
        bug_paper_check.append((url, int(paper[5])))
        continue
      if int(paper[5]) > 2022:
        continue
      year_set.add(int(paper[5]))
    if len(year_set) == 0:
      continue
      # year_first, year_last = 2023, 2022
      # bug_check.append(scholar['url'])
    year_last = max(year_set)
    year_dict[year_last] += 1
  return year_dict

In [ ]:
get_lastyear_dict(cluster7_idx_list)

In [ ]:
import pandas as pd
df_scholar = pd.DataFrame.from_records(scholar_all)

In [ ]:
ml_label = [('artificial_intelligence', 20031),
 ('artifical_intelligence', 49),
 ('artificial_inteligence', 30),
 ('artificial_intelligent', 52),
 ('ai', 910),
 ('computational_intelligence', 411),
 ('machine_learning', 44384),
 ('pattern_recognition', 2533),
 ('ml', 25),
 ('applied_machine_learning', 82),
 ('neural_network', 161),
 ('artificial_neural_network', 35),
 ('deep_neural_network', 23),
 ('deep_learning', 7642),
 ('modelling', 59),
 ('machine_deep_learning', 57),
 ('machine_intelligence', 45),
 ('statistical_machine_learning', 37),
 ('generative_adversarial_networks', 22),
 ('generative_models', 108),
 ('explainable_ai', 149),
 ('xai', 30),
 ('ai_safety', 36),
 ('fairness', 42),
 ('algorithmic_fairness', 38),
 ('interpretability', 72),
 ('robustness', 26),
 ('domain_adaptation', 48),
 ('trustworthy_ai', 28),
 ('deep_reinforcement_learning', 61),
 ('representation_learning', 145),
 ('neural_networks', 835),
 ('artificial_neural_networks', 100),
 ('deep_neural_networks', 73),
 ('graph_neural_networks', 83),
 ('recurrent_neural_networks', 26),
 ('convolutional_neural_networks', 24),
 ('spiking_neural_networks', 24),
 ('automl', 54),
 ('support_vector_machines', 22),
 ('gaussian_processes', 41),
 ('kernel_methods', 42),
 ('few_shot_learning', 21),
 ('statistical_learning', 89),
 ('adversarial_learning', 28),
 ('metric_learning', 25),
 ('manifold_learning', 23),
 ('ensemble_learning', 22),
 ('transfer_learning', 132),
 ('meta_learning', 53),
 ('multimodal_learning', 45),
 ('unsupervised_learning', 92),
 ('online_learning', 57),
 ('active_learning', 56),
 ('federated_learning', 116),
 ('self_supervised_learning', 46),
 ('adversarial_machine_learning', 45),
 ('imitation_learning', 44),
 ('continual_learning', 41),
 ('semi_supervised_learning', 32),
 ('graph_representation_learning', 31),
 ('classification', 94),
 ('pattern_classification', 26)]

cv_label = [('computer_vision', 16777),
 ('vision', 82),
 ('machine_vision', 177),
 ('3d_computer_vision', 40),
 ('3d_reconstruction', 148),
 ('imaging', 73),
 ('3d_vision', 73),
 ('3d_modeling', 21),
 ('3d', 21),
 ('machine_perception', 39),
 ('3d_perception', 22),
 ('perception', 105),
 ('visual_perception', 47),
 ('robot_perception', 25),
 ('image_processing', 3664),
 ('computer_graphics', 1030),
 ('image_analysis', 342),
 ('computational_photography', 269),
 ('face_recognition', 118),
 ('image_processing', 3664),
 ('image_analysis', 342),
 ('medical_image_analysis', 758),
 ('biomedical_image_analysis', 103),
 ('medical_image_processing', 280),
 ('biomedical_image_processing', 50),
 ('digital_image_processing', 162),
 ('signal_and_image_processing', 113),
 ('medical_image_computing', 81),
 ('image_segmentation', 69),
 ('image_retrieval', 48),
 ('image_and_signal_processing', 47),
 ('image_reconstruction', 43),
 ('image_registration', 42),
 ('signal_image_processing', 34),
 ('image_understanding', 33),
 ('document_image_analysis', 32),
 ('image_restoration', 30),
 ('medical_image', 30),
 ('image_recognition', 28),
 ('image_guided_surgery', 26),
 ('image_classification', 24),
 ('image_enhancement', 23),
 ('image_compression', 21),
 ('handwriting_recognition', 44),
 ('face_analysis', 22),
 ('eye_tracking', 50),
 ('super_resolution', 28),
 ('visual_tracking', 30),
 ('scene_understanding', 48),
 ('ocr', 44),
 ('object_tracking', 44),
 ('visual_analytics', 75),
 ('image_and_video_processing', 117),
 ('image_and_video_processing', 117),
 ('image_video_processing', 55),
 ('video_coding', 76),
 ('video_processing', 86),
 ('video_understanding', 46),
 ('video_analysis', 41),
 ('video_surveillance', 40),
 ('video_analytics', 38),
 ('video_compression', 33)]

nlp_label = [ ('natural_language_processing', 6963),
 ('computational_linguistics', 1200),
 ('nlp', 818),
 ('text_mining', 602),
 ('linguistics', 146),
 ('text_analytics', 59),
 ('text_analysis', 33),
 ('natural_language_understanding', 116),
 ('text_generation', 22),
 ('parsing', 35),
 ('text_summarization', 35),
 ('summarization', 25),
 ('lexical_semantics', 30),
 ('text_classification', 30),
 ('semantics', 121),
 ('discourse', 20),
 ('language_technology', 59),
 ('natural_language_generation', 77),
 ('vision_and_language', 49),
 ('speech_and_language_processing', 30), 
 ('spoken_language_processing', 26),
 ('language_acquisition', 27),
 ('natural_language', 24),
 ('semantic_technologies', 22),
 ('discourse_analysis', 22),
 ('language_modeling', 39),
 ('commonsense_reasoning', 24),
 ('computational_semantics', 24),
 ('syntax', 37),
 ('psycholinguistics', 52),
 ('morphology', 29),
 ('machine_translation', 431),
 ('statistical_machine_translation', 28),
 ('neural_machine_translation', 32),
 ('sentiment_analysis', 240),
 ('opinion_mining', 53),
 ('argumentation', 53),
 ('affective_computing', 315),
 ('digital_humanities', 113),
 ('personalization', 60),
 ('user_modeling', 54),
 ('computational_social_science', 238),
 ('social_computing', 114),
 ('computational_social_choice', 34),
 ('social_media_mining', 25),
 ('social_media', 122),
 ('social_media_analysis', 50),
 ('social_media_analytics', 32),
 ('bionlp', 28),
 ('biomedical_text_mining', 21),
 ('document_analysis', 46),
 ('pragmatics', 32)]

information_retrieval_label = [('information_retrieval', 1266),
 ('search', 66),
 ('web_search', 37),
 ('multimedia_information_retrieval', 23),
 ('multimedia_retrieval', 38),
 ('information_extraction', 328),
 ('question_answering', 108),
 ('corpus_linguistics', 102)]

knowledge_graph_label = [('semantic_web', 458),
 ('ontologies', 97),
 ('knowledge_representation', 240),
 ('knowledge_graphs', 113),
 ('knowledge_graph', 110),
 ('knowledge_engineering', 87),
 ('knowledge_representation_and_reasoning', 85),
 ('automated_reasoning', 57),
 ('reasoning', 34),
 ('knowledge_management', 102),
 ('knowledge_discovery', 66)]

data_science_label = [('data_mining', 3908),
 ('datamining', 40),
 ('data_sciences', 26),
 ('data_science', 2821),
 ('information_science', 22),
 ('information_technology', 48),
 ('informatics', 47),
 ('data_engineering', 27),
 ('data_analysis', 378),
 ('big_data', 1138),
 ('big_data_analytics', 295),
 ('big_data_analysis', 59),
 ('bigdata', 39),
 ('data_analytics', 498)]

robotics_label = [('robotics', 3413),
 ('robotic', 46),
 ('intelligent_agents', 50),
 ('autonomous_agents', 21),
 ('intelligent_systems', 144),
 ('reinforcement_learning', 2047),
 ('multi_agent_systems', 364),
 ('multi_agent_system', 35),
 ('multiagent_systems', 151),
 ('mobile_robotics', 94),
 ('cognitive_robotics', 64),
 ('developmental_robotics', 31),
 ('social_robotics', 53),
 ('robot_learning', 71),
 ('robot_vision', 67),
 ('multi_robot_systems', 25),
 ('autonomous_robots', 23),
 ('evolutionary_robotics', 21),
 ('manipulation', 28),
 ('control_theory', 209),
 ('control', 268),
 ('optimal_control', 89),
 ('control_systems', 171),
 ('control_system', 45),
 ('fault_diagnosis', 64),
 ('automatic_control', 44),
 ('model_predictive_control', 41),
 ('intelligent_control', 39),
 ('controls', 38),
 ('process_control', 37),
 ('adaptive_control', 35),
 ('control_engineering', 35),
 ('nonlinear_control', 34),
 ('motor_control', 29),
 ('stochastic_control', 28),
 ('systems_and_control', 25),
 ('robust_control', 23)]

autonomous_driving_label = [('autonomous_driving', 277),
 ('autonomous_vehicles', 136),
 ('intelligent_vehicles', 37),
 ('autonomous_vehicle', 30),
 ('automated_driving', 28),
 ('automation', 128),
 ('autonomous_systems', 122)]

signal_processing_label = [('signal_processing', 1889),
 ('wireless_communications', 304),
 ('speech_processing', 305),
 ('speech_recognition', 404),
 ('automatic_speech_recognition', 83),
 ('speech_synthesis', 77),
 ('speech', 59),
 ('speaker_recognition', 50),
 ('speech_enhancement', 39),
 ('speech_signal_processing', 31),
 ('speech_technology', 35),
 ('digital_signal_processing', 178),
 ('statistical_signal_processing', 160),
 ('audio_signal_processing', 75),
 ('audio_processing', 40),
 ('audio', 30),
 ('time_series', 167),
 ('time_series_analysis', 146),
 ('time_series_forecasting', 42)]

biology_label = [('bioinformatics', 2321),
 ('biostatistics', 268),
 ('biomedical_image_analysis', 103),
 ('biomedical_imaging', 94),
 ('biometrics', 500),
 ('biomedical_engineering', 293),
 ('biomedical_signal_processing', 210),
 ('biomedical_informatics', 220),
 ('biomedical_image_processing', 50),
 ('biomedical', 37),
 ('biomedical_data_science', 26),
 ('biomechanics', 81),
 ('microbiome', 37),
 ('biomedicine', 21),
 ('computational_biology', 1056),
 ('mathematical_biology', 27),
 ('systems_biology', 246),
 ('structural_biology', 45),
 ('molecular_biology', 43),
 ('synthetic_biology', 41),
 ('biology', 35),
 ('microbiology', 25),
 ('conservation_biology', 22),
 ('clinical_informatics', 44),
 ('protein_engineering', 22),
 ('protein_design', 21),
 ('biophysics', 108),
 ('biosignal_processing', 34),
 ('bioengineering', 55),
 ('microbiome', 37),
 ('biotechnology', 37),
 ('structural_bioinformatics', 30),
 ('biosensors', 26),
 ('biochemistry', 26),
 ('genetics', 142),
 ('statistical_genetics', 48),
 ('next_generation_sequencing', 28),
 ('genomics', 386),
 ('epigenomics', 25),
 ('single_cell_genomics', 22),
 ('computational_genomics', 39),
 ('metagenomics', 39),
 ('population_genetics', 29),
 ('molecular_modeling', 29)]

medical_label = [('drug_discovery', 179),
 ('drug_design', 43),
 ('precision_medicine', 113),
 ('computational_medicine', 24),
 ('personalized_medicine', 43),
 ('medicine', 92),
 ('ontology', 90),
 ('medical_image_analysis', 758),
 ('medical_imaging', 940),
 ('medical_image_processing', 280),
 ('medical_image_computing', 81),
 ('medical_informatics', 204),
 ('medical_devices', 36),
 ('medical_robotics', 26),
 ('healthcare', 257),
 ('health_informatics', 248),
 ('healthcare_informatics', 32),
 ('digital_health', 141),
 ('structural_health_monitoring', 113),
 ('public_health', 76),
 ('mental_health', 42),
 ('mobile_health', 41),
 ('healthcare_analytics', 41),
 ('e_health', 38),
 ('ehealth', 37),
 ('health', 34),
 ('ai_in_healthcare', 33),
 ('mhealth', 32),
 ('electronic_health_records', 31),
 ('health_care', 29),
 ('prognostics_and_health_management', 24),
 ('ai_for_healthcare', 24),
 ('pharmacogenomics', 22),
 ('autism', 22),
 ('neurology', 21),
 ('epidemiology', 122),
 ('alzheimer%27s_disease', 22),
 ('infectious_disease', 21),
 ('cancer_biology', 40),
 ('cancer_genomics', 70),
 ('cancer', 103),
 ('immunology', 64),
 ('diabetes', 27),
 ('breast_cancer', 21),
 ('cancer_research', 30),
 ('clinical_trials', 30),
 ('diagnostics', 39),
 ('psychiatry', 49),
 ('radiation_oncology', 28),
 ('cardiology', 40),
 ('eeg', 85),
 ('radiology', 76),
 ('neurosurgery', 26),
 ('rehabilitation', 25),
 ('oncology', 68)]

neuroscience_label = [('computational_neuroscience', 542),
 ('cognitive_neuroscience', 121),
 ('neuroinformatics', 47),
 ('neuroscience', 679),
 ('neuroimaging', 236),
 ('theoretical_neuroscience', 40),
 ('mri', 95),
 ('fmri', 59),
 ('neural_engineering', 49)]

psychology_label = [('psychology', 91),
 ('cognitive_psychology', 48),
 ('cognitive_science', 431),
 ('cognitive_modeling', 40),
 ('cognitive_computing', 35),
 ('cognition', 53),
 ('cognitive_systems', 51),
 ('knowledge_acquisition', 20)]

general_cs_label = [('computer_science', 809),
 ('software_engineering', 845),
 ('software_development', 75),
 ('software_architecture', 38),
 ('software', 34),
 ('empirical_software_engineering', 22),
 ('computer_architecture', 214),
 ('computer_engineering', 27),
 ('computer_systems', 37),
 ('operating_systems', 40),
 ('soft_computing', 358),
 ('mobile_computing', 155),
 ('computer_networks', 178),
 ('computer_network', 37),
 ('computer_networking', 30),
 ('software_defined_networking', 23),
 ('wireless', 22),
 ('wireless_networking', 35),
 ('wireless_network', 21),
 ('wireless_networks', 135),
 ('communication_networks', 26),
 ('wireless_communication', 181),
 ('telecommunications', 30),
 ('communications', 64),
 ('algorithms', 425),
 ('theoretical_computer_science', 68),
 ('theory_of_computation', 22),
 ('programming_languages', 137),
 ('python', 32),
 ('logic_programming', 78),
 ('constraint_programming', 95),
 ('programming', 73),
 ('coding_theory', 41),
 ('compilers', 66),
 ('genetic_programming', 62),
 ('genetic_algorithms', 64),
 ('genetic_algorithm', 27),
 ('evolutionary_algorithms', 172),
 ('evolutionary_algorithm', 24),
 ('evolutionary_computation', 433),
 ('algorithm', 46),
 ('heuristic_search', 48),
 ('algorithmic_game_theory', 47),
 ('game_theory', 337),
 ('formal_methods', 114),
 ('formal_verification', 33),
 ('verification', 23),
 ('software_testing', 85),
 ('program_analysis', 39),
 ('program_synthesis', 26),
 ('fuzzy_logic', 197),
 ('computational_logic', 24),
 ('logic', 110),
 ('graph_theory', 130),
 ('graph_algorithms', 23),
 ('virtual_reality', 257),
 ('augmented_reality', 321),
 ('ar_vr', 31),
 ('mixed_reality', 50),
 ('databases', 189),
 ('database', 91),
 ('database_systems', 48),
 ('data_management', 78),
 ('high_performance_computing', 371),
 ('hpc', 100),
 ('distributed_systems', 363),
 ('distributed_computing', 133),
 ('distributed_algorithms', 30),
 ('parallel_computing', 147),
 ('parallel_and_distributed_computing', 22),
 ('parallel_processing', 41),
 ('parallel_algorithms', 23),
 ('parallel_programming', 27),
 ('distributed_system', 35),
 ('cloud_computing', 565),
 ('cloud', 32),
 ('gpu', 30),
 ('cyber_security', 322),
 ('cybersecurity', 285),
 ('security', 305),
 ('information_security', 225),
 ('network_security', 222),
 ('computer_security', 142),
 ('security_and_privacy', 49),
 ('multimedia_security', 25),
 ('hardware_security', 27),
 ('software_security', 22),
 ('web_security', 21),
 ('malware_analysis', 28),
 ('malware_detection', 20),
 ('data_privacy', 61),
 ('privacy', 209),
 ('differential_privacy', 62),
 ('surveillance', 21),
 ('cryptography', 146),
 ('blockchain', 231),
 ('blockchain_technology', 21),
 ('block_chain', 21)]

math_label = [('applied_mathematics', 201),
 ('mathematics', 147),
 ('combinatorics', 33),
 ('mathematical_modeling', 94),
 ('mathematical_modelling', 62),
 ('computational_mathematics', 35),
 ('mathematical_physics', 34),
 ('mathematical_programming', 28),
 ('geometry', 29),
 ('computational_geometry', 77),
 ('geometric_modeling', 24),
 ('partial_differential_equations', 29)]

physics_label = [('physics', 272),
 ('computational_physics', 110),
 ('astrophysics', 99),
 ('statistical_physics', 97),
 ('theoretical_physics', 70),
 ('condensed_matter_physics', 68),
 ('medical_physics', 61),
 ('nuclear_physics', 24),
 ('particle_physics', 88),
 ('plasma_physics', 28),
 ('dark_matter', 21),
 ('condensed_matter_theory', 28),
 ('string_theory', 27),
 ('high_energy_physics', 62),
 ('computational_fluid_dynamics', 76),
 ('fluid_mechanics', 47),
 ('fluid_dynamics', 38),
 ('microfluidics', 37),
 ('statistical_mechanics', 67),
 ('quantum_computing', 215),
 ('quantum_information', 73),
 ('quantum_physics', 55),
 ('quantum_computation', 46),
 ('quantum_machine_learning', 29),
 ('quantum_mechanics', 24),
 ('quantum_optics', 28)]

chemistry_label = [('physical_chemistry', 22),
 ('quantum_chemistry', 59),
 ('computational_chemistry', 146),
 ('cheminformatics', 119),
 ('chemistry', 51),
 ('chemoinformatics', 51),
 ('chemometrics', 45),
 ('chemical_engineering', 39),
 ('electrochemistry', 22),
 ('theoretical_chemistry', 21),
 ('catalysis', 47),
 ('metabolomics', 39)]

philosophy_label = [('philosophy', 50),
 ('ethics', 44),
 ('philosophy_of_science', 27),
 ('computational_creativity', 80)]

hci_label = [('human_computer_interaction', 748),
 ('human_machine_interaction', 52),
 ('intelligent_user_interfaces', 26),
 ('human_robot_interaction', 284),
 ('human_robot_collaboration', 21),
 ('brain_computer_interface', 153),
 ('hci', 174),
 ('user_experience', 30),
 ('data_visualization', 140),
 ('information_visualization', 74),
 ('scientific_visualization', 24),
 ('visualization', 126)]

education_label = [('e_learning', 107),
 ('elearning', 21),
 ('learning_analytics', 70),
 ('educational_technology', 62),
 ('intelligent_tutoring_systems', 44),
 ('educational_data_mining', 61),
 ('technology_enhanced_learning', 29),
 ('computer_science_education', 55),
 ('engineering_education', 35),
 ('education', 105)]

iot_label = [('remote_sensing', 728),
 ('iot', 568),
 ('internet_of_things', 543),
 ('internet_of_things_iot', 45),
 ('ubiquitous_computing', 98),
 ('embedded_systems', 243),
 ('embedded_system', 78),
 ('wearables', 37),
 ('intelligent_transportation_system', 28)]

energy_sustainability_label = [('climate_change', 84),
 ('climate_science', 22),
 ('climate', 21),
 ('water_resources', 24),
 ('renewable_energy', 217),
 ('energy', 121),
 ('energy_efficiency', 58),
 ('energy_systems', 35),
 ('energy_storage', 35),
 ('energy_harvesting', 29),
 ('solar_energy', 28),
 ('energy_management', 22),
 ('wind_energy', 21),
 ('computational_sustainability', 24),
 ('sustainability', 67)]

recommendation_label = [('recommender_systems', 459),
 ('recommender_system', 104),
 ('recommendation_systems', 75),
 ('recommendation_system', 50),
 ('recommendation', 36)]

statistics_label = [('bayesian_statistics', 210),
 ('probabilistic_inference', 25),
 ('statistical_inference', 57),
 ('bayesian_inference', 135),
 ('variational_inference', 24),
 ('monte_carlo_methods', 24),
 ('bayesian_methods', 31),
 ('statistical_modeling', 107),
 ('statistical_modelling', 35),
 ('statistics', 1285),
 ('computational_statistics', 76),
 ('applied_statistics', 75),
 ('statistical_analysis', 53),
 ('probability', 52),
 ('probability_theory', 53),
 ('probabilistic_models', 27),
 ('probabilistic_modeling', 30),
 ('high_dimensional_statistics', 34),
 ('applied_probability', 34),
 ('stochastic_processes', 36),
 ('information_theory', 408),
 ('causal_inference', 218),
 ('causality', 79),
 ('graphical_models', 83),
 ('probabilistic_graphical_models', 60),
 ('bayesian_networks', 57),
 ('optimization', 1924),
 ('optimisation', 90),
 ('convex_optimization', 88),
 ('mathematical_optimization', 54),
 ('combinatorial_optimization', 119),
 ('multi_objective_optimization', 63),
 ('stochastic_optimization', 72),
 ('bayesian_optimization', 55),
 ('numerical_optimization', 54),
 ('optimization_algorithms', 46),
 ('distributed_optimization', 41),
 ('optimization_techniques', 37),
 ('discrete_optimization', 27),
 ('multiobjective_optimization', 27),
 ('network_optimization', 24),
 ('optimization_theory', 22),
 ('learning_theory', 59),
 ('statistical_learning_theory', 40)]

network_science_label = [('social_network', 45),
 ('social_networks', 176),
 ('social_network_analysis', 219),
 ('network_science', 161),
 ('network_analysis', 81),
 ('networking', 145),
 ('networks', 143),
 ('complex_networks', 143),
 ('complex_systems', 174),
 ('complex_network', 31),
 ('adaptive_systems', 31),
 ('complex_adaptive_systems', 21)]

business_label = [('supply_chain_management', 66),
 ('supply_chain', 25),
 ('finance', 110),
 ('quantitative_finance', 66),
 ('computational_finance', 30),
 ('financial_engineering', 23),
 ('fintech', 49),
 ('mathematical_finance', 22),
 ('computational_advertising', 41),
 ('e_commerce', 41),
 ('marketing', 37),
 ('entrepreneurship', 38),
 ('economics', 58),
 ('econometrics', 83),
 ('algorithmic_trading', 22),
 ('asset_pricing', 22),
 ('smart_manufacturing', 30),
 ('innovation', 54),
 ('business_intelligence', 100),
 ('decision_support_systems', 92),
 ('business_analytics', 90),
 ('business_process_management', 32),
 ('industrial_engineering', 22),
 ('project_management', 50),
 ('smart_city', 39),
 ('smart_cities', 87),
 ('e_government', 27),
 ('industry_4.0', 80),
 ('intelligent_transportation', 25),
 ('intelligent_transportation_systems', 110)]

domain_labels = [ml_label, cv_label, nlp_label, information_retrieval_label, knowledge_graph_label, data_science_label, robotics_label, autonomous_driving_label, 
signal_processing_label, biology_label, medical_label, neuroscience_label, psychology_label, general_cs_label, math_label, physics_label, 
chemistry_label, philosophy_label, hci_label, education_label, iot_label, energy_sustainability_label, recommendation_label, statistics_label, 
network_science_label, business_label]
domain_label_t = ['ml_label', 'cv_label', 'nlp_label', 'information_retrieval_label', 'knowledge_graph_label', 'data_science_label', 'robotics_label', 'autonomous_driving_label', 
'signal_processing_label', 'biology_label', 'medical_label', 'neuroscience_label', 'psychology_label', 'general_cs_label', 'math_label', 'physics_label', 
'chemistry_label', 'philosophy_label', 'hci_label', 'education_label', 'iot_label', 'energy_sustainability_label', 'recommendation_label', 'statistics_label', 
'network_science_label', 'business_label']

In [ ]:
for idx, label in enumerate(domain_labels):
  female = 0
  male = 0
  for index, row in df_scholar.iterrows():
    for nlp_item in label:
      if nlp_item[0] in row["domain_labels"]:
        if row["gender"] == 'M':
          male += 1 / len(row['domain_labels'])
        if row["gender"] == 'F':
          female += 1 / len(row['domain_labels'])
  # print(male)
  # print(female)
  print(f'{domain_label_t[idx]}: {male}, {female}, {female/(male+female)}')